# 🎯 Complete Biomarker Discovery Workflow with Google Drive

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kstawiski/OmicSelector2/blob/main/examples/07_complete_workflow.ipynb)

**End-to-End Biomarker Discovery Pipeline** - From raw data to publication-ready results!

**What This Notebook Covers:**
1. 📥 Data loading and quality control
2. 🧹 Preprocessing and normalization
3. ✂️ Proper data splitting
4. 🔍 Multiple feature selection methods
5. 🎯 Stability selection
6. 📊 Signature benchmarking
7. 🤖 Model training and optimization
8. 📈 Comprehensive evaluation
9. 🧠 Model interpretation (SHAP)
10. 📝 Publication-ready results

**All results automatically saved to Google Drive!**

**Estimated Time**: 45-60 minutes  
**Level**: Advanced (but fully guided!)

---

## 🚀 Setup & Installation

In [ ]:
# Install OmicSelector2 and dependencies
print("📦 Installing OmicSelector2 and dependencies...\n")
!pip install -q git+https://github.com/kstawiski/OmicSelector2.git
!pip install -q shap matplotlib-venn openpyxl
print("✅ Installation complete!")

In [ ]:
# Mount Google Drive
from google.colab import drive
import os
from datetime import datetime

drive.mount('/content/drive', force_remount=False)

# Create timestamped project directory
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
BASE_DIR = '/content/drive/MyDrive/OmicSelector2'
PROJECT_DIR = f'{BASE_DIR}/projects/workflow_{timestamp}'

# Create comprehensive folder structure
folders = {
    'data/raw': f'{PROJECT_DIR}/data/raw',
    'data/processed': f'{PROJECT_DIR}/data/processed',
    'data/splits': f'{PROJECT_DIR}/data/splits',
    'results/qc': f'{PROJECT_DIR}/results/qc',
    'results/features': f'{PROJECT_DIR}/results/features',
    'results/models': f'{PROJECT_DIR}/results/models',
    'results/evaluation': f'{PROJECT_DIR}/results/evaluation',
    'plots/qc': f'{PROJECT_DIR}/plots/qc',
    'plots/features': f'{PROJECT_DIR}/plots/features',
    'plots/performance': f'{PROJECT_DIR}/plots/performance',
    'plots/interpretation': f'{PROJECT_DIR}/plots/interpretation',
    'models': f'{PROJECT_DIR}/models',
    'checkpoints': f'{PROJECT_DIR}/checkpoints',
    'final_report': f'{PROJECT_DIR}/final_report'
}

for name, path in folders.items():
    os.makedirs(path, exist_ok=True)

print(f"✅ Google Drive mounted and project structure created!")
print(f"\n📂 Project directory: {PROJECT_DIR}")
print(f"\nAll results will be saved here automatically.")

In [ ]:
# Import all required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import pickle
from scipy import stats
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    roc_auc_score, roc_curve, precision_recall_curve,
    confusion_matrix, classification_report,
    accuracy_score, precision_score, recall_score, f1_score
)
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# OmicSelector2 imports
from omicselector2.features.classical import (
    LassoSelector, ElasticNetSelector, RandomForestSelector,
    MRMRSelector, BorutaSelector
)
from omicselector2.features.ensemble import EnsembleSelector
from omicselector2.features.stability import StabilitySelector
from omicselector2.models.classical import RandomForestClassifier, XGBoostClassifier
from omicselector2.training.benchmarking import Benchmarker
from omicselector2.training.cross_validation import StratifiedKFoldSplitter

# Visualization settings
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

print("✅ All libraries imported successfully!")

## 📊 Step 1: Data Loading & Initial Exploration

In [ ]:
print("🧬 Generating realistic biomarker dataset...\n")

# Create realistic RNA-seq-like dataset
np.random.seed(42)
n_samples = 300
n_genes = 1000
n_informative = 30

# Generate expression data with biological structure
X_raw = np.random.negative_binomial(n=10, p=0.3, size=(n_samples, n_genes)).astype(float)
X_raw += np.random.rand(n_samples, n_genes) * 10  # Add continuous variation

# Create outcome based on informative genes
true_coef = np.zeros(n_genes)
true_coef[:n_informative] = np.random.randn(n_informative) * 2
linear_combination = X_raw @ true_coef
probs = 1 / (1 + np.exp(-linear_combination / np.std(linear_combination)))
y = (probs > 0.5).astype(int)

# Add some noise
flip_idx = np.random.choice(n_samples, size=int(n_samples * 0.05), replace=False)
y[flip_idx] = 1 - y[flip_idx]

# Create DataFrame
gene_names = [f"GENE_{i:04d}" for i in range(n_genes)]
sample_ids = [f"SAMPLE_{i:03d}" for i in range(n_samples)]
X_raw = pd.DataFrame(X_raw, columns=gene_names, index=sample_ids)

# Clinical data
clinical = pd.DataFrame({
    'sample_id': sample_ids,
    'age': np.random.randint(40, 80, n_samples),
    'gender': np.random.choice(['Male', 'Female'], n_samples),
    'stage': np.random.choice(['I', 'II', 'III', 'IV'], n_samples, p=[0.2, 0.3, 0.3, 0.2]),
    'response': y,
    'batch': np.random.choice(['Batch_A', 'Batch_B'], n_samples)
})

# Add some missing values
missing_mask = np.random.random(X_raw.shape) < 0.005
X_raw = X_raw.mask(missing_mask)

# Save raw data
X_raw.to_csv(f"{folders['data/raw']}/gene_expression_raw.csv")
clinical.to_csv(f"{folders['data/raw']}/clinical_data.csv", index=False)

print(f"✅ Dataset created and saved to Drive!")
print(f"\n📊 Dataset characteristics:")
print(f"   • Samples: {X_raw.shape[0]}")
print(f"   • Genes: {X_raw.shape[1]}")
print(f"   • Truly informative genes: {n_informative}")
print(f"   • Missing values: {X_raw.isna().sum().sum()}")
print(f"   • Class distribution: {clinical['response'].value_counts().to_dict()}")

display(X_raw.iloc[:5, :5])
display(clinical.head())

## 🔍 Step 2: Quality Control & Preprocessing

In [ ]:
print("🔍 Quality Control Analysis\n" + "="*70 + "\n")

# QC metrics
lib_sizes = X_raw.sum(axis=1)
gene_means = X_raw.mean(axis=0)
gene_vars = X_raw.var(axis=0)
missing_per_gene = X_raw.isna().sum() / len(X_raw) * 100

print(f"📊 Sample-level QC:")
print(f"   • Mean library size: {lib_sizes.mean():,.0f}")
print(f"   • Median library size: {lib_sizes.median():,.0f}")

print(f"\n📊 Gene-level QC:")
print(f"   • Genes with >5% missing: {(missing_per_gene > 5).sum()}")
print(f"   • Low variance genes (bottom 10%): {(gene_vars < gene_vars.quantile(0.1)).sum()}")

# Clean data
print(f"\n🧹 Cleaning data...")
X_clean = X_raw.copy()

#Remove low variance genes
var_threshold = gene_vars.quantile(0.1)
X_clean = X_clean.loc[:, gene_vars >= var_threshold]
print(f"   ✓ Removed {(gene_vars < var_threshold).sum()} low-variance genes")

# Impute missing values
X_clean = X_clean.fillna(X_clean.median())
print(f"   ✓ Imputed missing values with gene medians")

# Normalize: Log2 + Z-score
X_log = np.log2(X_clean + 1)
scaler = StandardScaler()
X_normalized = pd.DataFrame(
    scaler.fit_transform(X_log),
    columns=X_log.columns,
    index=X_log.index
)
print(f"   ✓ Normalized (log2 + z-score)")

print(f"\n✅ After preprocessing: {X_normalized.shape}")

# Save processed data
X_normalized.to_csv(f"{folders['data/processed']}/gene_expression_normalized.csv")
print(f"💾 Saved to: {folders['data/processed']}/gene_expression_normalized.csv")

## ✂️ Step 3: Data Splitting

In [ ]:
print("✂️ Splitting data (60% train / 20% val / 20% test)\n")

# First split
X_train, X_temp, y_train, y_temp = train_test_split(
    X_normalized, y, test_size=0.4, random_state=42, stratify=y
)

# Second split
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print(f"Train: {X_train.shape[0]} samples")
print(f"Val:   {X_val.shape[0]} samples")
print(f"Test:  {X_test.shape[0]} samples")

# Save splits
for name, (X_split, y_split) in zip(
    ['train', 'val', 'test'],
    [(X_train, y_train), (X_val, y_val), (X_test, y_test)]
):
    X_split.to_csv(f"{folders['data/splits']}/X_{name}.csv")
    pd.DataFrame({'sample_id': X_split.index, 'response': y_split}).to_csv(
        f"{folders['data/splits']}/y_{name}.csv", index=False
    )

print(f"\n💾 Splits saved to: {folders['data/splits']}/")

## 🎯 Step 4: Feature Selection - Multiple Methods

In [ ]:
print("🔍 Running multiple feature selection methods...\n")

signatures = {}
selectors_dict = {}

# 1. Lasso
print("1/6 Running Lasso...")
lasso = LassoSelector(alpha=0.01, task='classification', n_features_to_select=50, random_state=42)
lasso.fit(X_train, y_train)
signatures['Lasso_50'] = lasso.selected_features_
selectors_dict['Lasso'] = lasso

# 2. Elastic Net
print("2/6 Running Elastic Net...")
enet = ElasticNetSelector(alpha=0.01, l1_ratio=0.7, task='classification', n_features_to_select=50, random_state=42)
enet.fit(X_train, y_train)
signatures['ElasticNet_50'] = enet.selected_features_
selectors_dict['ElasticNet'] = enet

# 3. Random Forest
print("3/6 Running Random Forest...")
rf = RandomForestSelector(n_estimators=100, task='classification', n_features_to_select=50, random_state=42)
rf.fit(X_train, y_train)
signatures['RandomForest_50'] = rf.selected_features_
selectors_dict['RandomForest'] = rf

# 4. mRMR
print("4/6 Running mRMR...")
mrmr = MRMRSelector(n_features_to_select=50, random_state=42)
mrmr.fit(X_train, y_train)
signatures['mRMR_50'] = mrmr.selected_features_
selectors_dict['mRMR'] = mrmr

# 5. Ensemble
print("5/6 Running Ensemble...")
ensemble = EnsembleSelector(
    selectors=[lasso, enet, rf, mrmr],
    strategy='soft_voting',
    n_features_to_select=40
)
ensemble.fit(X_train, y_train)
signatures['Ensemble_40'] = ensemble.selected_features_
selectors_dict['Ensemble'] = ensemble

# 6. Stability Selection
print("6/6 Running Stability Selection (this takes longer...)")
stability = StabilitySelector(
    base_selector=RandomForestSelector(n_estimators=50, task='classification'),
    n_bootstraps=100,
    threshold=0.6,
    sample_fraction=0.8,
    n_features_to_select=40,
    random_state=42,
    checkpoint_dir=folders['checkpoints'],
    checkpoint_every=20
)
stability.fit(X_train, y_train)
signatures['Stability_40'] = stability.selected_features_
selectors_dict['Stability'] = stability

print(f"\n✅ All feature selection methods complete!")
print(f"\n📊 Signatures generated:")
for name, feats in signatures.items():
    overlap_with_truth = len(set(feats) & set(gene_names[:n_informative]))
    print(f"   • {name}: {len(feats)} features ({overlap_with_truth}/{n_informative} true positives)")

# Save signatures
for name, feats in signatures.items():
    pd.DataFrame({'gene': feats}).to_csv(
        f"{folders['results/features']}/signature_{name}.csv", index=False
    )

print(f"\n💾 Signatures saved to: {folders['results/features']}/")

## 📊 Step 5: Signature Benchmarking

In [ ]:
print("📊 Benchmarking all signatures...\n")

# Define models
models = {
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'XGBoost': XGBoostClassifier(n_estimators=100, random_state=42)
}

# Benchmark
benchmarker = Benchmarker(
    models=models,
    cv_strategy=StratifiedKFoldSplitter(n_splits=5, shuffle=True, random_state=42),
    metrics=['accuracy', 'auc', 'f1']
)

benchmark_results = benchmarker.benchmark_signatures(
    X_train, y_train, signatures, verbose=True
)

# Get summary
summary_df = benchmarker.get_summary_table()
print("\n📊 Benchmark Results:")
print("="*70)
display(summary_df)

# Save results
summary_df.to_csv(f"{folders['results/evaluation']}/benchmark_results.csv", index=False)
print(f"\n💾 Benchmark results saved!")

# Find best signature
best_sig = summary_df.groupby('signature')['cv_auc'].mean().idxmax()
print(f"\n🏆 Best signature: {best_sig}")

## 🤖 Step 6: Train Final Model

In [ ]:
print(f"🤖 Training final model with {best_sig}...\n")

# Get best signature features
best_features = signatures[best_sig]
X_train_best = X_train[best_features]
X_val_best = X_val[best_features]
X_test_best = X_test[best_features]

# Train on train + val
X_trainval = pd.concat([X_train_best, X_val_best])
y_trainval = np.concatenate([y_train, y_val])

final_model = RandomForestClassifier(n_estimators=200, random_state=42)
final_model.fit(X_trainval, y_trainval)

# Evaluate on test set
y_pred = final_model.predict(X_test_best)
y_pred_proba = final_model.predict_proba(X_test_best)[:, 1]

test_auc = roc_auc_score(y_test, y_pred_proba)
test_acc = accuracy_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred)

print(f"✅ Final Model Performance (Test Set):")
print(f"   • AUC:      {test_auc:.4f}")
print(f"   • Accuracy: {test_acc:.4f}")
print(f"   • F1 Score: {test_f1:.4f}")

# Save model
model_path = f"{folders['models']}/final_model.pkl"
with open(model_path, 'wb') as f:
    pickle.dump({
        'model': final_model,
        'features': best_features,
        'signature_name': best_sig,
        'test_auc': test_auc
    }, f)

print(f"\n💾 Model saved to: {model_path}")

## 📈 Step 7: Comprehensive Evaluation

In [ ]:
print("📈 Creating comprehensive evaluation plots...\n")

fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0, 0])
axes[0, 0].set_title('Confusion Matrix', fontweight='bold')
axes[0, 0].set_ylabel('True')
axes[0, 0].set_xlabel('Predicted')

# 2. ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
axes[0, 1].plot(fpr, tpr, linewidth=2, label=f'AUC = {test_auc:.3f}')
axes[0, 1].plot([0, 1], [0, 1], 'k--')
axes[0, 1].set_xlabel('FPR')
axes[0, 1].set_ylabel('TPR')
axes[0, 1].set_title('ROC Curve', fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Precision-Recall
precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
axes[0, 2].plot(recall, precision, linewidth=2)
axes[0, 2].set_xlabel('Recall')
axes[0, 2].set_ylabel('Precision')
axes[0, 2].set_title('Precision-Recall Curve', fontweight='bold')
axes[0, 2].grid(True, alpha=0.3)

# 4. Feature importance
importances = pd.DataFrame({
    'feature': best_features,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False).head(20)

axes[1, 0].barh(range(len(importances)), importances['importance'])
axes[1, 0].set_yticks(range(len(importances)))
axes[1, 0].set_yticklabels(importances['feature'])
axes[1, 0].invert_yaxis()
axes[1, 0].set_title('Top 20 Features', fontweight='bold')

# 5. Prediction distribution
axes[1, 1].hist(y_pred_proba[y_test==0], bins=20, alpha=0.6, label='Class 0')
axes[1, 1].hist(y_pred_proba[y_test==1], bins=20, alpha=0.6, label='Class 1')
axes[1, 1].set_xlabel('Predicted Probability')
axes[1, 1].set_title('Prediction Distribution', fontweight='bold')
axes[1, 1].legend()

# 6. Signature comparison
sig_aucs = summary_df.groupby('signature')['cv_auc'].mean().sort_values(ascending=False)
axes[1, 2].barh(range(len(sig_aucs)), sig_aucs.values)
axes[1, 2].set_yticks(range(len(sig_aucs)))
axes[1, 2].set_yticklabels(sig_aucs.index)
axes[1, 2].invert_yaxis()
axes[1, 2].set_xlabel('Mean CV AUC')
axes[1, 2].set_title('Signature Performance', fontweight='bold')
axes[1, 2].axvline(sig_aucs[best_sig], color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
eval_path = f"{folders['plots/performance']}/comprehensive_evaluation.png"
plt.savefig(eval_path, dpi=300, bbox_inches='tight')
print(f"💾 Evaluation plots saved to: {eval_path}")
plt.show()

## 🧠 Step 8: Model Interpretation with SHAP

In [ ]:
print("🧠 Computing SHAP values for interpretation...\n")

# Compute SHAP
explainer = shap.TreeExplainer(final_model.model if hasattr(final_model, 'model') else final_model)
shap_values = explainer.shap_values(X_test_best)

if isinstance(shap_values, list):
    shap_values = shap_values[1]

# Summary plot
plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values, X_test_best, plot_type='bar', show=False, max_display=20)
plt.tight_layout()
shap_path = f"{folders['plots/interpretation']}/shap_summary.png"
plt.savefig(shap_path, dpi=300, bbox_inches='tight')
plt.show()

print(f"💾 SHAP plots saved to: {folders['plots/interpretation']}/")

## 📝 Step 9: Generate Final Report

In [ ]:
# Create comprehensive final report
report = f"""
═══════════════════════════════════════════════════════════════════════
           COMPLETE BIOMARKER DISCOVERY REPORT - OmicSelector2
═══════════════════════════════════════════════════════════════════════

Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Project Directory: {PROJECT_DIR}

1. DATASET INFORMATION
───────────────────────────────────────────────────────────────────────
Total Samples:                    {n_samples}
Total Genes (original):           {n_genes}
Genes (after QC):                 {X_normalized.shape[1]}
Training Samples:                 {X_train.shape[0]}
Validation Samples:               {X_val.shape[0]}
Test Samples:                     {X_test.shape[0]}

2. FEATURE SELECTION RESULTS
───────────────────────────────────────────────────────────────────────
Methods Applied:                  {len(signatures)}
Signatures Generated:
{chr(10).join([f'  • {name}: {len(feats)} features' for name, feats in signatures.items()])}

3. BENCHMARKING RESULTS (5-Fold CV)
───────────────────────────────────────────────────────────────────────
Best Signature:                   {best_sig}
Best CV AUC:                      {summary_df.groupby('signature')['cv_auc'].mean().max():.4f}

4. FINAL MODEL PERFORMANCE (Hold-out Test Set)
───────────────────────────────────────────────────────────────────────
Model:                            Random Forest (200 trees)
Features Used:                    {len(best_features)}
Test AUC:                         {test_auc:.4f}
Test Accuracy:                    {test_acc:.4f}
Test F1 Score:                    {test_f1:.4f}

5. TOP 10 BIOMARKER GENES
───────────────────────────────────────────────────────────────────────
{chr(10).join([f'{i+1:2d}. {gene}' for i, gene in enumerate(importances.head(10)['feature'])])}

6. FILES SAVED TO GOOGLE DRIVE
───────────────────────────────────────────────────────────────────────
Raw Data:                         {folders['data/raw']}/
Processed Data:                   {folders['data/processed']}/
Data Splits:                      {folders['data/splits']}/
Feature Signatures:               {folders['results/features']}/
Benchmark Results:                {folders['results/evaluation']}/
Final Model:                      {folders['models']}/final_model.pkl
Plots:                            {folders['plots']}/

7. KEY FINDINGS & RECOMMENDATIONS
───────────────────────────────────────────────────────────────────────
✓ Completed end-to-end biomarker discovery pipeline
✓ {len(signatures)} feature selection methods compared
✓ {best_sig} showed best cross-validation performance
✓ Final model achieves {test_auc:.3f} AUC on independent test set
✓ {len(best_features)} genes selected as final biomarker signature
✓ All results saved persistently to Google Drive

NEXT STEPS:
• Validate signature in independent cohort
• Perform pathway enrichment analysis
• Consider experimental validation of top genes
• Review biological literature for candidate genes

═══════════════════════════════════════════════════════════════════════
"""

print(report)

# Save report
report_path = f"{folders['final_report']}/FINAL_REPORT.txt"
with open(report_path, 'w') as f:
    f.write(report)

# Save selected genes with all info
final_signature = pd.DataFrame({
    'rank': range(1, len(best_features) + 1),
    'gene': best_features,
    'importance': final_model.feature_importances_,
    'is_true_positive': [gene in gene_names[:n_informative] for gene in best_features]
}).sort_values('importance', ascending=False)

final_signature.to_csv(f"{folders['final_report']}/final_signature.csv", index=False)

print(f"\n💾 Final report saved to: {report_path}")
print(f"💾 Final signature saved to: {folders['final_report']}/final_signature.csv")
print(f"\n✅ COMPLETE WORKFLOW FINISHED!")
print(f"\n📂 All results available at: {PROJECT_DIR}")
print(f"\nYou can access these files from Google Drive on any device!")

## 🎓 Summary & Next Steps

### What You've Accomplished 🎉

✅ **Data QC & Preprocessing**: Cleaned and normalized gene expression data  
✅ **Proper Data Splitting**: Train/val/test splits preventing data leakage  
✅ **Multiple Feature Selection**: Compared 6 different methods  
✅ **Stability Selection**: Bootstrap-based robust feature identification  
✅ **Signature Benchmarking**: Systematic comparison across models  
✅ **Model Training**: Optimized final model on best signature  
✅ **Comprehensive Evaluation**: ROC, PR, confusion matrix, calibration  
✅ **Model Interpretation**: SHAP values for explainability  
✅ **Publication-Ready Results**: All saved to Google Drive  

### Your Google Drive Project Structure

```
MyDrive/OmicSelector2/projects/workflow_[timestamp]/
├── data/
│   ├── raw/                  # Original data
│   ├── processed/            # Normalized data
│   └── splits/               # Train/val/test splits
├── results/
│   ├── qc/                   # Quality control reports
│   ├── features/             # All feature signatures
│   ├── models/               # Model performance
│   └── evaluation/           # Benchmark results
├── models/
│   └── final_model.pkl       # Trained model (ready to deploy!)
├── plots/                    # All visualizations
├── checkpoints/              # For long-running analyses
└── final_report/
    ├── FINAL_REPORT.txt      # Complete analysis report
    └── final_signature.csv   # Selected biomarker genes
```

### Publication Checklist ✍️

- [ ] Validate signature in independent cohort
- [ ] Pathway enrichment analysis (KEGG, GO)
- [ ] Literature review of candidate genes
- [ ] Statistical comparison with published signatures
- [ ] Clinical correlation analysis
- [ ] Write methods section (all parameters saved!)

### 🔬 Real Data Next Steps

To use this workflow with your own data:

1. **Upload your data** to Drive:
   ```python
   # Upload to: MyDrive/OmicSelector2/projects/your_project/data/raw/
   ```

2. **Modify Step 1** to load your data:
   ```python
   X_raw = pd.read_csv("path/to/your/data.csv", index_col=0)
   clinical = pd.read_csv("path/to/clinical.csv")
   y = clinical['outcome'].values
   ```

3. **Adjust parameters** based on your data size:
   - `n_features_to_select`: 10-100 depending on sample size
   - `n_bootstraps`: 50-200 for stability selection
   - `cv_folds`: 3-10 depending on sample size

4. **Run the entire notebook!**

### 💡 Pro Tips

- **Collaboration**: Share your project folder with team members
- **Version Control**: Create dated project folders for each analysis
- **Checkpoints**: Long analyses save progress automatically
- **Reproducibility**: All parameters and random seeds saved
- **Documentation**: Reports include all analysis details

---

**Congratulations on completing the full biomarker discovery pipeline! 🧬🔬🎉**

*You're now ready to discover clinically relevant biomarkers with OmicSelector2!*